In [47]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [48]:

df = pd.read_csv("/kaggle/input/refined-dataset/data.csv")
df

,Country,Year,Status,Life_Expectancy,Adult_Mortality,Infant_Deaths,Alcohol,Percentage_Exp,HepatitisB,BMI,...,winsorized_Polio,winsorized_Tot_Exp,winsorized_Diphtheria,winsorized_HIV,winsorized_GDP,winsorized_Population,winsorized_thinness_1to19_years,winsorized_thinness_5to9_years,winsorized_Income_Comp_Of_Resources,winsorized_Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,19.1,...,47.0,8.16,65.0,0.1,584.259210,15799542.0,14.2,14.4,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,18.6,...,58.0,8.18,62.0,0.1,612.696514,327582.0,14.2,14.4,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,18.1,...,62.0,8.13,64.0,0.1,631.744976,15799542.0,14.2,14.4,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,17.6,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,14.2,14.4,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,17.2,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,14.2,14.4,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2909,Zimbabwe,2004,Developing,44.3,723.0,27,4.36,0.000000,68.0,27.1,...,67.0,7.13,65.0,2.3,454.366654,12777511.0,9.4,9.4,0.407,9.2
2910,Zimbabwe,2003,Developing,44.5,715.0,26,4.06,0.000000,7.0,26.7,...,47.0,6.52,68.0,2.3,453.351155,12633897.0,9.8,9.9,0.418,9.5
2911,Zimbabwe,2002,Developing,44.8,73.0,25,4.43,0.000000,73.0,26.3,...,73.0,6.53,71.0,2.3,57.348340,125525.0,1.2,1.3,0.427,10.0
2912,Zimbabwe,2001,Developing,45.3,686.0,25,1.72,0.000000,76.0,25.9,...,76.0,6.16,75.0,2.3,548.587312,12366165.0,1.6,1.7,0.427,9.8


In [49]:
numerical_columns = [x for x in list(df.columns) if x.startswith("winsorized")]
numerical_columns

['winsorized_Life_Expectancy',
 'winsorized_Adult_Mortality',
 'winsorized_Infant_Deaths',
 'winsorized_Alcohol',
 'winsorized_Percentage_Exp',
 'winsorized_HepatitisB',
 'winsorized_Under_Five_Deaths',
 'winsorized_Polio',
 'winsorized_Tot_Exp',
 'winsorized_Diphtheria',
 'winsorized_HIV',
 'winsorized_GDP',
 'winsorized_Population',
 'winsorized_thinness_1to19_years',
 'winsorized_thinness_5to9_years',
 'winsorized_Income_Comp_Of_Resources',
 'winsorized_Schooling']

In [50]:
numerical_columns = list(set(numerical_columns) - set("winsorized_Life_Expectancy"))
y = "winsorized_Life_Expectancy"
X_train = np.array(df[numerical_columns])
y_train = np.array(df[y])
prediction = np.array(df[numerical_columns].iloc[0].values)

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,
                                   random_state=42, 
                                   test_size=0.25, 
                                   shuffle=True)

In [42]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
tscv = TimeSeriesSplit(n_splits=5)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    for mf in np.linspace(100, 150, 6):
        for ne in np.linspace(50, 100, 6):
            for md in np.linspace(20, 40, 5):
                for msl in np.linspace(30, 100, 8):
                    rfr = RandomForestRegressor(
                        max_features=int(mf/100),
                        n_estimators=int(ne),
                        max_depth=int(md),
                        min_samples_leaf=int(msl))
                    rfr.fit(X_tr, y_tr)
                    score.append([i,
                                  mf, 
                                  ne,
                                  md, 
                                  msl, 
                                  rfr.score(X_val, y_val), r2_score(rfr.predict(X_val ) , y_val)])
    i += 1

KeyboardInterrupt: 

In [10]:
score[0]
max(score , key = lambda x : x[5])

[4, 110.0, 60.0, 25.0, 30.0, 0.8656955516433953]

we get the highest accuracy for max_features = 1.1 , n_estimators = 60.0 , max_depth = 25.0 , min_samples_leaf = 30.0

In [ ]:
model = RandomForestRegressor(  max_features=int(110.0/100),
                        n_estimators=int(60.0),
                        max_depth=int(25.0),
                        min_samples_leaf=int(30.0))

In [13]:
from sklearn.metrics import r2_score
max_score = 0
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = RandomForestRegressor(  max_features=int(110.0/100),
        n_estimators=int(60.0),
        max_depth=int(25.0),
        min_samples_leaf=int(30.0))
    rfr.fit(X_tr, y_tr)
    score.append([i,
                  mf, 
                  ne,
                  md, 
                  msl, 
                  rfr.score(X_val, y_val)])
    if rfr.score(X_val, y_val) > max_score:
        model = rfr
                
                

In [39]:
rfr.predict([prediction])

array([62.38690081])

In [46]:
import pickle
pickle.dump(rfr, open('model_baseline_RandomForestRegressor.pkl','wb'))